# Measurement Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Measurement kata](./Measurements.ipynb). Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Measurements kata (and this workbook):

1.	Basic linear algebra
2.	The concept of qubit and multi-qubit systems
3.	Single-qubit and multi-qubit quantum gates and using them to manipulate the state of the system

To begin, first prepare this notebook for execution (if you skip the first step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get "Invalid test name" error):

In [ ]:
%package Microsoft.Quantum.Katas::0.11.2003.3107

In [ ]:
%workspace reload

## Part I. Discriminating Orthogonal States

### Task 1.1. $|0\rangle$ or $|1\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|0\rangle$ or the $|1\rangle$ state.

**Output:**  `true` if the qubit was in the $|1\rangle$ state, or `false` if it was in the $|0\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

The input qubit is guaranteed to be either in basis state $|0\rangle$ or $|1\rangle$. This means that when measuring the qubit in the Pauli $Z$ basis (the computational basis), the measurement will report the input state without any doubt.  

In Q# the operation [`M()`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.m) can be used to measure a single qubit in the Pauli $Z$ basis. The measurement result is a value of type `Result`: the operation `M` will return `One` if the input qubit was in the $|1\rangle$ state and `Zero` if the input qubit was in the $|0\rangle$ state. Since we need to encode the first case as `true` and the second one as `false`, we can return the result of equality comparison between measurement result and `One`.

In [ ]:
%kata T101_IsQubitOne_Test 

operation IsQubitOne (q : Qubit) : Bool {
    return M(q) == One;
}

[Return to task 1.1 of the Measurements kata.](./Measurements.ipynb#Task-1.1.-$|0\rangle$-or-$|1\rangle$?)

### Task 1.2. Set the qubit to the $|0\rangle$ state.

**Input:** A qubit in an arbitrary state.

**Goal:**  Change the state of the qubit to $|0\rangle$.

### Solution

A fundamental postulate of quantum computing says that when we measure a qubit in a possible superposition state, it will collapse to the state that corresponds to the outcome of the measurement. This means that regardless of the original qubit state, after we measure the qubit in the Pauli $Z$ basis, it will end up in the $|0\rangle$ or $|1\rangle$ state. 

After we use the operation `M()` to measure the input qubit in the Pauli $Z$ basis, there are two possibilities:
1.	The qubit collapses to the state $|0\rangle$ (measurement outcome `Zero`), and we don’t need to change anything.
2.	The qubit collapses to the state $|1\rangle$ (measurement outcome `One`), and we need to flip the state of the qubit. This can be done with the [**X** gate](../tutorials/SingleQubitGates/SingleQubitGates.ipynb#Pauli-Gates).

In [ ]:
%kata T102_InitializeQubit_Test 

operation InitializeQubit (q : Qubit) : Unit {
    if (M(q) == One) {
        X(q);
    }
}

[Return to task 1.2 of the Measurements kata.](./Measurements.ipynb#Task-1.2.-Set-the-qubit-to-the-$|0\rangle$-state.)

### Task 1.3. $|+\rangle$ or $|-\rangle$?

**Input:** A qubit which is guaranteed to be in either the $|+\rangle$ or the $|-\rangle$ state. As a reminder, $|+\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big)$, $|-\rangle = \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big)$.

**Output:** `true` if the qubit was in the $|+\rangle$ state, or `false` if it was in the $|-\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

Both input state are superposition states, with equal absolute values of amplitudes of both basis states. This means if the sate is measured in the Pauli $Z$ basis, like we did in the previous task, there is a 50-50 chance of measuring `One` or `Zero`, which won't give us the necessary information.  

To determine in which state the input qubit is with certainty, we want to transform the qubit into a state where there is no superposition with respect to the basis in which we perform the measurement. 

Consider how we can prepare the input states, starting with basis states: $H|0\rangle = |+\rangle$ and $H|1\rangle = |-\rangle$. 
This transformation can also be undone by applying the **H** gate again (remember that the **H** gate is self-adjoint, i.e., it equals its own inverse): $H|+\rangle = |0\rangle$  and $H|-\rangle = |1\rangle$.

Once we have the $|0\rangle$ or $|1\rangle$ state, we can use the same principle as in task 1.1 to measure the state and report the outcome. Note that in this task return value `true` corresponds to input state $|+\rangle$, so we compare the measurement result with `Zero`.

In [ ]:
%kata T103_IsQubitPlus_Test 

operation IsQubitPlus (q : Qubit) : Bool {
    H(q);
    return M(q) == Zero;
}

Another possible solution could be to measure in the Pauli $X$ basis (${|+\rangle, |-\rangle}$ basis), this means a transformation with the **H** gate before measurement is not needed. Again, measurement result `Zero` would correspond to state $|+\rangle$.

In Q#, measuring in another Pauli basis can be done with the [`Measure()`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) operation.

In [ ]:
%kata T103_IsQubitPlus_Test 

operation IsQubitPlus (q : Qubit) : Bool {
   return Measure([PauliX], [q]) == Zero;
}

[Return to task 1.3 of the Measurements kata.](./Measurements.ipynb#Task-1.3.-$|+\rangle$-or-$|-\rangle$?)

### Task 1.4. $|A\rangle$ or $|B\rangle$?

**Inputs:** 

1. Angle $\alpha$, in radians, represented as a `Double`.
2. A qubit which is guaranteed to be in either the $|A\rangle$ or the $|B\rangle$ state, where $|A\rangle = \cos \alpha |0\rangle + \sin \alpha |1\rangle$ and $|B\rangle = - \sin \alpha |0\rangle + \cos \alpha |1\rangle$.

**Output:** `true` if the qubit was in the $|A\rangle$ state, or `false` if it was in the $|B\rangle$ state. The state of the qubit at the end of the operation does not matter.

### Solution

We take a similar approach to the previous task: figure out a way to prepare the input states from the basis states and apply adjoint of that preparation before measuring the qubit.

To create the input states $|A\rangle$ and $|B\rangle$, a [**Ry**](../tutorials/SingleQubitGates/SingleQubitGates.ipynb#Rotation-Gates) gate with $\theta= 2\alpha$ was applied to the basis states $|0\rangle$ and $|1\rangle$. As a reminder, 

$$R_y = \begin{bmatrix} \cos\frac{\theta}{2} & - \sin\frac{\theta}{2} \\ \sin\frac{\theta}{2} & \cos\frac{\theta}{2} \end{bmatrix}$$

We can return the inputs state to the basis sates by applying **Ry** gate with $-2 \alpha$ as the rotation angle parameter to the input qubit.

The measurement in Pauli $Z$ basis gives two possibilities:
1. The qubit is measured as $|1\rangle$, the input state was $|B\rangle$, we return `false`.
2. The qubit is measured as $|0\rangle$, the input state was $|A\rangle$, we return `true`.

In [ ]:
%kata T104_IsQubitA_Test

operation IsQubitA (alpha : Double, q : Qubit) : Bool {
    Ry(-2.0 * alpha, q);
    return M(q) == Zero;
}

[Return to task 1.4 of the Measurements kata.](./Measurements.ipynb#Task-1.4.-$|A\rangle$-or-$|B\rangle$?)

### Task 1.5. $|00\rangle$ or $|11\rangle$?

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in either the $|00\rangle$ or the $|11\rangle$ state.

**Output:** 0 if the qubits were in the $|00\rangle$ state, or 1 if they were in the $|11\rangle$ state. The state of the qubits at the end of the operation does not matter.

### Solution

Both qubits in the input array are in the same state: for $|00\rangle$ each individual qubit is in state $|0\rangle$, for $|11\rangle$ each individual qubit is in state $|1\rangle$. Therefore, if we measure one qubit we will know the state of the other qubit.  
In other words, if the first qubit measures as `One`, we know that the qubits in the input array are in state $|11\rangle$, and if it measures as `Zero`, we know they are in state $|00\rangle$.

> `condition ? trueValue | falseValue` is Q#'s ternary operator: it returns `trueValue` if `condition` is true and `falseValue` otherwise.

In [ ]:
%kata T105_ZeroZeroOrOneOne_Test

operation ZeroZeroOrOneOne (qs : Qubit[]) : Int {
    return M(qs[0]) == One ? 1 | 0;
}

[Return to task 1.5 of the Measurements kata.](./Measurements.ipynb#Task-1.5.-$|00\rangle$-or-$|11\rangle$?)

### Task 1.6. Distinguish four basis states.

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four basis states ($|00\rangle$, $|01\rangle$, $|10\rangle$, or $|11\rangle$).

**Output:**

* 0 if the qubits were in the $|00\rangle$ state,
* 1 if they were in the $|01\rangle$ state, 
* 2 if they were in the $|10\rangle$ state, 
* 3 if they were in the $|11\rangle$ state.

In this task and the subsequent ones the order of qubit states in task description matches the order of qubits in the array (i.e., $|10\rangle$ state corresponds to `qs[0]` in state $|1\rangle$ and `qs[1]` in state $|0\rangle$).

The state of the qubits at the end of the operation does not matter.

### Solution

Unlike in the previous task, this time measuring the first qubit won't give us any information on the second qubit, so we need to measure both qubits.

First, we measure both qubits in the input array and store the result in `m1` and `m2`. We can decode these results like this:  
- `m1` is $|0\rangle$ and `m2` is $|0\rangle$: we return $0\cdot2+0 = 0$
- `m1` is $|0\rangle$ and `m2` is $|1\rangle$: we return $0\cdot2+1 = 1$
- `m1` is $|1\rangle$ and `m2` is $|0\rangle$: we return $1\cdot2+0 = 2$
- `m1` is $|1\rangle$ and `m2` is $|1\rangle$: we return $1\cdot2+1 = 3$

In other words, we treat the measurement results as the binary notation of the return value in [big endian notation](../tutorials/MultiQubitSystems/MultiQubitSystems.ipynb#Endianness).

In [ ]:
%kata T106_BasisStateMeasurement_Test

operation BasisStateMeasurement (qs : Qubit[]) : Int {
    // Measurement on the first qubit gives the higher bit of the answer, on the second - the lower
    let m1 = M(qs[0]) == Zero ? 0 | 1;
    let m2 = M(qs[1]) == Zero ? 0 | 1;
    return m1 * 2 + m2;
}

[Return to task 1.6 of the Measurements kata.](./Measurements.ipynb#Task-1.6.-Distinguish-four-basis-states.)

### Task 1.7. Distinguish two basis states given by bit strings

**Inputs:** 

1. $N$ qubits (stored in an array of length $N$) which are guaranteed to be in one of the two basis states described by the given bit strings.
2. Two bit strings represented as `Bool[]`s.

**Output:** 

* 0 if the qubits were in the basis state described by the first bit string,
* 1 if they were in the basis state described by the second bit string.

Bit values `false` and `true` correspond to $|0\rangle$ and $|1\rangle$ states. You are guaranteed that both bit strings have the same length as the qubit array, and that the bit strings differ in at least one bit.

You can use exactly one measurement. The state of the qubits at the end of the operation does not matter.

> Example:  for bit strings `[false, true, false]` and `[false, false, true]` return 0 corresponds to state $|010\rangle$, and return 1 corresponds to state $|001\rangle$.

### Solution

To solve this task we will use two steps. Like many other programming languages, Q# allows you to write functions to make code more readable and reusable. 

The first step is to find first bit that differs between bit strings `bit1` and `bit2`. For that we define a function `FindFirstDiff()` which loops through both `Bool[]`s and returns the first index where the bit strings differ. 

In [ ]:
function FindFirstDiff (bits1 : Bool[], bits2 : Bool[]) : Int {
    for (i in 0 .. Length(bits1) - 1) {
        if (bits1[i] != bits2[i]) {
            return i;
        }
    }
    return -1;
}

The second step is implementing the main operation: once we have found the first different bit, we measure the qubit in the corresponding position to see whether it is in state $|0\rangle$ or $|1\rangle$. If it is in state $|0\rangle$, `res` takes the value `false`, if it is in state $|1\rangle$ it takes the value `true`. 

`res == bits1[firstDiff]` compares the measurement result with the bit of `bits1` in the differing position. This effectively checks if the qubits are in the basis state described by the first or by the second bit string. 
The two possible outcomes are: 
1. The qubits are in the state described by the first bit string; then `res` will be equal to `bits1[firstDiff]` and the method will return `0`. 
2. The qubits are in the state described by the second bit string; then `res` will be not equal to `bits1[firstDiff]` (we know it has to be equal to `bits2[firstDiff]` which does not equal `bits1[firstDiff]`), and the method will return `1`. 

In [ ]:
%kata T107_TwoBitstringsMeasurement_Test

operation TwoBitstringsMeasurement (qs : Qubit[], bits1 : Bool[], bits2 : Bool[]) : Int {
    // find the first index at which the bit strings are different and measure it
    let firstDiff = FindFirstDiff(bits1, bits2);
    let res = M(qs[firstDiff]) == One;

    return res == bits1[firstDiff] ? 0 | 1;
}

[Return to task 1.7 of the Measurements kata.](./Measurements.ipynb#Task-1.7.-Distinguish-two-basis-states-given-by-bit-strings)

### Task 1.8. Distinguish two superposition states given by two arrays of bit strings - 1 measurement

**Inputs:** 

1. $N$ qubits (stored in an array of length $N$) which are guaranteed to be in one of the two superposition states described by the given arrays of bit strings.
2. Two arrays of bit strings represented as `Bool[][]`s.  
The arrays have dimensions $M_1 \times N$ and $M_2 \times N$ respectively, where $N$ is the number of qubits and $M_1$ and $M_2$ are the numbers of bit strings in each array. Note that in general $M_1 \neq M_2$.  
An array of bit strings `[b₁, ..., bₘ]` defines a state that is an equal superposition of all basis states defined by bit strings $b_1, ..., b_m$.  
For example, an array of bit strings `[[false, true, false], [false, true, true]]` defines a superposition state $\frac{1}{\sqrt2}\big(|010\rangle + |011\rangle\big)$.

You are guaranteed that there exists an index of a qubit Q for which: 
 - all the bit strings in the first array have the same value in this position (all `bits1[j][Q]` are the same),
 - all the bit strings in the second array have the same value in this position (all `bits2[j][Q]` are the same),
 - these values are different for the first and the second arrays.

> For example, for arrays `[[false, true, false], [false, true, true]]` and `[[true, false, true], [false, false, true]]` return 0 corresponds to state $\frac{1}{\sqrt2}\big(|010\rangle + |011\rangle\big)$, return 1 - to state $\frac{1}{\sqrt2}\big(|101\rangle + |001\rangle\big)$, and you can distinguish these states perfectly by measuring the second qubit.

**Output:** 

* 0 if qubits were in the superposition state described by the first array,
* 1 if they were in the superposition state described by the second array.

The state of the qubits at the end of the operation does not matter.

You are allowed to use exactly one measurement.  


### Solution

Like the previous solution we are looking for the difference between the two bit strings. In the `FindFirstSuperpositionDiff()` we search for an index which have 2 properties:
1. The value at the index in both bit strings are `true` or `false`. If this is not the case, you cannot be sure that the measurement always return the same result. For example, given this state: $\frac{1}{\sqrt2}\big(|010\rangle + |011\rangle\big)$ if you measure the third qubit, you will get $|0\rangle$ $50\%$ of the time, therefore you want to either use index 1 or 2 for the measurement.
2. At the found index the bit strings in the different arrays cannot be similar.

In [ ]:
function FindFirstSuperpositionDiff (bits1 : Bool[][], bits2 : Bool[][], Nqubits : Int) : Int {
    for (i in 0 .. Nqubits - 1) {
        // count the number of 1s in i-th position in bit strings of both arrays
        mutable val1 = 0;
        mutable val2 = 0;
        for (j in 0 .. Length(bits1) - 1) {
            if (bits1[j][i]) {
                set val1 += 1;
            }
        }
        for (k in 0 .. Length(bits2) - 1) {
            if (bits2[k][i]) {
                set val2 += 1;
            }
        }
        if ((val1 == Length(bits1) and val2 == 0) or (val1 == 0 and val2 == Length(bits2))) {
            return i;
        }
    }
    return -1;
}

The second step is very similar to the previous exercise, given the index of the different bit in the string we measure the qubit on that position.  
Here the library function `ResultAsBool()` function is used, this functions return `true` if the measurement result is `One` and `false` if the result is `Zero`. This library function can replace the `M(qs[firstDiff]) == One` that was used before.

In [ ]:
%kata T108_SuperpositionOneMeasurement_Test
open Microsoft.Quantum.Convert;

operation SuperpositionOneMeasurement (qs : Qubit[], bits1 : Bool[][], bits2 : Bool[][]) : Int {
    let diff = FindFirstSuperpositionDiff(bits1, bits2, Length(qs));

    let res = ResultAsBool(M(qs[diff]));

    if (res == bits1[0][diff]) {
        return 0;
    }
    else {
        return 1;
    }
}

[Return to task 1.8 of the Measurements kata.](./Measurements.ipynb#Task-1.8.-Distinguish-two-superposition-states-given-by-two-arrays-of-bit-strings---1-measurement)

### Task 1.9. Distinguish two superposition states given by two arrays of bit strings

**Inputs:** 

1. $N$ qubits (stored in an array of length $N$) which are guaranteed to be in one of the two superposition states described by the given arrays of bit strings.
2. Two arrays of bit strings represented as `Bool[][]`s.  
The arrays describe the superposition states in the same way as in the previous task,
i.e. they have dimensions $M_1 \times N$ and $M_2 \times N$ respectively, $N$ being the number of qubits.

The only constraint on the bit strings is that **all bit strings in the two arrays are distinct**. 

> Example:  for bit strings `[[false, true, false], [false, false, true]]` and `[[true, true, true], [false, true, true]]` return 0 corresponds to state $\frac{1}{\sqrt2}\big(|010\rangle + |001\rangle\big)$, return 1 to state $\frac{1}{\sqrt2}\big(|111\rangle + |011\rangle\big)$.

**Output:** 

* 0 if qubits were in the superposition state described by the first array,
* 1 if they were in the superposition state described by the second array.

The state of the qubits at the end of the operation does not matter.

You can use as many measurements as you wish.  


### Solution

Because all the bit strings are guaranteed to be different we can use an easier solution then before. We make use of two build in library functions. Firstly, we measure all the qubits in the array using `MultiM()`, this function will result in an array with results. For example when we apply `MultiM()` on the state $|01\rangle$ the result will be `[Zero,One]`.  
To make this array easier to compare to the input bit strings we convert this array into an integer using the [`ResultArrayAsInt()`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.convert.resultarrayasint) method, this will create an integer using the little endian format.  
To check if the measured result equals a bit string in `bits1` or `bits2`, we also convert those bit strings to an integer in the little endian format using the `BoolArrayAsInt()` function.  
Now that we have two integers we can easily find if the qubits are represented by either bit strings. 

In [ ]:
%kata T109_SuperpositionMeasurement_Test
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

operation SuperpositionMeasurement (qs : Qubit[], bits1 : Bool[][], bits2 : Bool[][]) : Int {
    let measuredState = ResultArrayAsInt(MultiM(qs));
    for (s in bits1) {
        if (BoolArrayAsInt(s) == measuredState) {
            return 0;
        }
    }
    return 1;
}

[Return to task 1.9 of the Measurements kata.](./Measurements.ipynb#Task-1.9.-Distinguish-two-superposition-states-given-by-two-arrays-of-bit-strings)

### Task 1.10. $|0...0\rangle$ state or W state ?

**Input:** $N$ qubits (stored in an array of length $N$) which are guaranteed to be either in the $|0...0\rangle$ state or in the [W state](https://en.wikipedia.org/wiki/W_state). 

**Output:**

* 0 if the qubits were in the $|0...0\rangle$ state,
* 1 if they were in the W state.

The state of the qubits at the end of the operation does not matter.

### Solution

The $W$ state looks as follows: $\frac{1}{\sqrt{3}}(|001\rangle+|010\rangle+|100\rangle)$. This means that in the $W$ state there is always one qubit in the $|1\rangle$ state while in the $|0..0\rangle$ state there are no qubits in this state.  
As solution we will count the number of qubits in the $|1\rangle$ state if this is one we have a $W$ state, if it is zero we know it is the $|0..0\rangle$ state.

In [ ]:
%kata T110_AllZerosOrWState_Test

operation AllZerosOrWState (qs : Qubit[]) : Int {
    mutable countOnes = 0;

    for (q in qs) {
        if (M(q) == One) {
            set countOnes += 1;
        }
    }
    
    if(countOnes > 0){
        return 1; // The number of One's counted are higher then 0 this means it must be W state.
    }else{
        return 0;
    }
}

In the previous Task we seen the use of the `MultiM()` and `ResultArrayAsInt()` functions. These can also be used in this function to make the solution shorter.

In [ ]:
%kata T110_AllZerosOrWState_Test
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Measurement;

operation AllZerosOrWState_Alternate (qs : Qubit[]) : Int {
    return ResultArrayAsInt(MultiM(qs)) == 0 ? 0 | 1;
}

[Return to task 1.10 of the Measurements kata.](./Measurements.ipynb#Task-1.10.-$|0...0\rangle$-state-or-W-state-?)

### Task 1.11. GHZ state or W state ?

**Input:** $N \ge 2$ qubits (stored in an array of length $N$) which are guaranteed to be either in the [GHZ state](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state) or in the [W state](https://en.wikipedia.org/wiki/W_state).

**Output:** 

* 0 if the qubits were in the GHZ state,
* 1 if they were in the W state.

The state of the qubits at the end of the operation does not matter.

### Solution

The $GHZ$ state looks as follows $|GHZ\rangle = \frac{|000\rangle + |111\rangle}{\sqrt{2}}$.  
As seen in the previous task the $W$ state will have one qubit in the $|1\rangle$ state. The $GHZ$ state will either have all qubits in the $|1\rangle$ state or all qubits in the $|0\rangle$ state.  
This means if you find one qubit in state $|1\rangle$ you must have $W$ if you measure something else it must be the $GHZ$ state.

In [ ]:
%kata T111_GHZOrWState_Test

operation GHZOrWState (qs : Qubit[]) : Int {
    mutable countOnes = 0;

    for (q in qs) {
        if (M(q) == One) {
            set countOnes += 1;
        }
    }
    if(countOnes == 1){
        return 1; 
    }else{
        return 0;
    }
}

[Return to task 1.11 of the Measurements kata.](./Measurements.ipynb#Task-1.11.-GHZ-state-or-W-state-?)

### Task 1.12. Distinguish four Bell states.

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four Bell states.

**Output:**

* 0 if they were in the state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big)$,
* 1 if they were in the state $|\Phi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle - |11\rangle\big)$,
* 2 if they were in the state $|\Psi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle + |10\rangle\big)$,
* 3 if they were in the state $|\Psi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle - |10\rangle\big)$.

The state of the qubits at the end of the operation does not matter.

### Solution

Firstly, let's take a look how a Bell state is created, let's start with the $|00\rangle$ basis state. To start we apply the **H** gate on the first qubit which gives $H|00\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}|0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |10\rangle)$, lastly appling the **CNOT** gate creates the bell sate $CNOT\frac{1}{\sqrt{2}}(|00\rangle + |10\rangle) = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle) = |\Phi^{+}\rangle$.  

Once the qubits are entangled in the Bell state you can't use individual measurement to separate the states. Therefore, we want to convert the Bell states back to the basis states, so we can use the methods we applied before to separate the states.  
To bring the $|\Phi^{+}\rangle$ Bell state back to the basis state $|00\rangle$ you can apply the same gates which have been applied to create the state in reverse order (so first **CNOT** then **H**).

> A bit more background on creating all the Bell states can be found in the appendix of [this](https://techcommunity.microsoft.com/t5/educator-developer-blog/a-beginner-8217-s-guide-to-quantum-computing-and-q/ba-p/380212) blog post.

In [ ]:
%kata T112_BellState_Test

operation BellState (qs : Qubit[]) : Int {
    CNOT(qs[0], qs[1]);
    H(qs[0]);

    // these changes brought the state back to one of the 2-qubit basis states from task 1.6 (but in different order)
    let m1 = M(qs[0]) == Zero ? 0 | 1;
    let m2 = M(qs[1]) == Zero ? 0 | 1;
    return m2 * 2 + m1;
}

[Return to task 1.12 of the Measurements kata.](./Measurements.ipynb#Task-1.12.-Distinguish-four-Bell-states.)

### Task 1.13. Distinguish four orthogonal 2-qubit states.

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four orthogonal states.

**Output:**

* 0 if they were in the state $|S_0\rangle = \frac{1}{2} \big(|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$,
* 1 if they were in the state $|S_1\rangle = \frac{1}{2} \big(|00\rangle - |01\rangle + |10\rangle - |11\rangle\big)$,
* 2 if they were in the state $|S_2\rangle = \frac{1}{2} \big(|00\rangle + |01\rangle - |10\rangle - |11\rangle\big)$,
* 3 if they were in the state $|S_3\rangle = \frac{1}{2} \big(|00\rangle - |01\rangle - |10\rangle + |11\rangle\big)$.

The state of the qubits at the end of the operation does not matter.

### Solution

Like the previous task let's take a look if the states can be converted back to the basis states from [task 1.6](#Task-1.6.-Distinguish-four-basis-states.).  
Starting with the first state $|S_0\rangle$, we can create this state from the $|00\rangle$ state by appling an **H** gate to each qubit. Let's take a look how this works, we start by creating the compound gate of two **H** gates (for background on this check the [Multi Qubit Workbook](../tutorials/MultiQubitGates/Workbook_MultiQubitGates.ipynb#Exercise-1:-Compound-Gate)):  
$$ H \otimes H = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \otimes \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} = \frac{1}{2} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{bmatrix}  $$

Appling this tho the basis sate $|00\rangle$ gives:
$$\frac{1}{2} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{bmatrix} \cdot \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \end{bmatrix} = \frac{1}{2}\begin{bmatrix}1 \\ 1 \\ 1 \\ 1 \end{bmatrix} = \frac{1}{2} \big(|00\rangle + |01\rangle + |10\rangle + |11\rangle\big) = |S_0\rangle$$
Knowing how we created the input state we can convert the states back to the basis state like done in the previous task.  

As you can see where are leveraging the `BasisStateMeasurement()` function which we have written in task 1.6, to reduce code writing.
> If you get the error `No identifier with the name "BasisStateMeasurement" exists.` you first have to execute the task 1.6 cell.

In [ ]:
%kata T113_TwoQubitState_Test

operation TwoQubitState (qs : Qubit[]) : Int {
    H(qs[0]);
    H(qs[1]);
    return BasisStateMeasurement(qs);
}

[Return to task 1.13 of the Measurements kata.](./Measurements.ipynb#Task-1.13.-Distinguish-four-orthogonal-2-qubit-states.)

### Task 1.14*. Distinguish four orthogonal 2-qubit states, part 2.

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four orthogonal states.

**Output:**

* 0 if they were in the state $|S_0\rangle = \frac{1}{2} \big(+ |00\rangle - |01\rangle - |10\rangle - |11\rangle\big)$,
* 1 if they were in the state $|S_1\rangle = \frac{1}{2} \big(- |00\rangle + |01\rangle - |10\rangle - |11\rangle\big)$,
* 2 if they were in the state $|S_2\rangle = \frac{1}{2} \big(- |00\rangle - |01\rangle + |10\rangle - |11\rangle\big)$,
* 3 if they were in the state $|S_3\rangle = \frac{1}{2} \big(- |00\rangle - |01\rangle - |10\rangle + |11\rangle\big)$.

The state of the qubits at the end of the operation does not matter.

### Solution

Here we are leveraging the same method as used in the previous exercise.  However, these input states are not directly created from the basis states, but from the Bell sates. The input states can be created by applying the **H** gate on the second qubit of the Bell states.   
To get back to the basis states we first transform back to the bell states and then transform to the basis states.

In [ ]:
%kata T114_TwoQubitStatePartTwo_Test

operation TwoQubitStatePartTwo (qs : Qubit[]) : Int {
    H(qs[1]);

    CNOT(qs[0], qs[1]);
    H(qs[0]);

    let m1 = M(qs[0]) == One ? 0 | 1;
    let m2 = M(qs[1]) == One ? 0 | 1;
    return m2 * 2 + m1;
}

[Return to task 1.14 of the Measurements kata.](./Measurements.ipynb#Task-1.14*.-Distinguish-four-orthogonal-2-qubit-states,-part-2.)

### Task 1.15**. Distinguish two orthogonal states on three qubits.

**Input:** Three qubits (stored in an array of length 3) which are guaranteed to be in one of the two orthogonal states.

**Output:**

* 0 if they were in the state $|S_0\rangle = \frac{1}{\sqrt{3}} \big(|100\rangle + \omega |010\rangle + \omega^2 |001\rangle \big)$,
* 1 if they were in the state $|S_1\rangle = \frac{1}{\sqrt{3}} \big(|100\rangle + \omega^2 |010\rangle + \omega |001\rangle \big)$.

Here $\omega = e^{2i \pi/ 3}$.

The state of the qubits at the end of the operation does not matter.

### Solution

We first apply a unitary operation to the input state so that it maps the first state to the W-state $\frac{1}{\sqrt{3}} \big( |100\rangle + |010\rangle + |001\rangle \big ) $.
This can be accomplished by a tensor product of the form $I_2 \otimes R \otimes R^2$, where
- $I_2$ denotes the 2x2 identity matrix which is applied to qubit 0,
- $R$ is the matrix $\begin{bmatrix} e^{-2i \pi / 3} & 0 \\ 0 & e^{2i \pi / 3} \end{bmatrix} $ which is applied to qubit 1,
- $R^2$ = $\begin{bmatrix} e^{-4i \pi / 3} & 0 \\ 0 & e^{4i \pi / 3} \end{bmatrix} $ which is applied to qubit 2.
> Note that upon applying the operator $I_2 \otimes R \otimes R^2$, the second state gets then mapped to $ \frac{1}{\sqrt{3}} \big ( |100\rangle + \omega |010\rangle + \omega^2 |001\rangle \big) $.

We can now perfectly distinguish these two states by invoking the inverse of the state prep routine for W-states which will map the first state to thestate $|000\rangle$ and the second state to some state which is guaranteed to be perpendicular to the state $|000\rangle$, i.e., the second state gets mapped to a superposition that does not involve $|000\rangle$.

In [ ]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;

operation WState_Arbitrary (qs : Qubit[]) : Unit is Adj + Ctl {
    let N = Length(qs);

    if (N == 1) {
        // base case of recursion: |1⟩
        X(qs[0]);
    } else {
        // |W_N⟩ = |0⟩|W_(N-1)⟩ + |1⟩|0...0⟩
        // do a rotation on the first qubit to split it into |0⟩ and |1⟩ with proper weights
        // |0⟩ -> sqrt((N-1)/N) |0⟩ + 1/sqrt(N) |1⟩
        let theta = ArcSin(1.0 / Sqrt(IntAsDouble(N)));
        Ry(2.0 * theta, qs[0]);

        // do a zero-controlled W-state generation for qubits 1..N-1
        X(qs[0]);
        Controlled WState_Arbitrary(qs[0 .. 0], qs[1 .. N - 1]);
        X(qs[0]);
    }
}

In [ ]:
%kata T115_ThreeQubitMeasurement_Test
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arithmetic;

operation ThreeQubitMeasurement (qs : Qubit[]) : Int {

    // rotate qubit 1 by angle - 2π/3 around z-axis
    Rz((4.0 * PI()) / 3.0, qs[1]);

    // rotate qubit 2 by angle - 4π/3 around z-axis
    Rz((8.0 * PI()) / 3.0, qs[2]);

    // Apply inverse state prep of 1/sqrt(3) ( |100⟩ + |010⟩ + |001⟩ )
    Adjoint WState_Arbitrary(qs);

    // measure all qubits: if all of them are 0, we have the first state,
    // if at least one of them is 1, we have the second state
    return MeasureInteger(LittleEndian(qs)) == 0 ? 0 | 1;
}

The solutions to the rest of the tasks are included in the [Measurements Workbook, Part 2](./Workbook_Measurements_Part2.ipynb).